# This notebook focuses on how to attach kfs_private bucket to the component

In [1]:
from kfp import dsl
from kfp.components import create_component_from_func
import kfp
from typing import NamedTuple

## Function creation
- Here @create_component_from_func decorator is used to convert save_file and read_data fucntion into components

In [2]:
@create_component_from_func
def save_file(data_path:str, data:str) -> NamedTuple('Outputs', [('file_loc', str)]):
    file_location = f"{data_path}/file.txt"
    with open(file_location, 'w') as f:
        f.write(data)
    return ([file_location])

@create_component_from_func
def read_data(file_location: str):
    with open(file_location, 'r') as f:
        print(f.read())

        

## Pipeline creation
- Here in the function `/kfs_private` is the name of the bucket that can be attached using `add_pvolumes` and the `pvc` value can be found from `katonic file manager`
    

In [3]:
@dsl.pipeline(
    name='Attach Volume Storage',
    description='A pipeline with volume storage'
)
def volume_pipeline():
    log_folder = "/kfs_private"
    data = "Hi!, I'm Bijoy and i love video games"

    save_file_task = save_file(log_folder, data).add_pvolumes({ log_folder:dsl.PipelineVolume(pvc="private-storage-f517"), })
    read_data(save_file_task.outputs['file_loc']).add_pvolumes({ log_folder:dsl.PipelineVolume(pvc="private-storage-f517"), })
    

In [4]:
from datetime import datetime
import uuid
EXPERIMENT_NAME = "Volume pipeline"
pipeline_func = volume_pipeline
pipeline_filename = pipeline_func.__name__ + f'{uuid.uuid1()}.pipeline.yaml'
kfp.compiler.Compiler().compile(pipeline_func, pipeline_filename)
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)
run_name = pipeline_func.__name__ + str(datetime.now().strftime("%d-%m-%Y-%H-%M-%S"))
client.upload_pipeline(pipeline_filename)
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename)